In [208]:
import sqlite3
from sqlite3 import Error
import json

In [209]:
import sqlite3
from sqlite3 import Error


def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn


def create_client_periods_table(conn):
    create_table_sql="""CREATE TABLE IF NOT EXISTS clients_periods (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    client_id TEXT NOT NULL,
    period_end TEXT NOT NULL,
    attribution INTEGER,
    total_customers INTEGER,
    contribution_rate REAL
);"""
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)


def create_records_table(conn):
    create_table_sql = """
    CREATE TABLE IF NOT EXISTS records (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        source TEXT NOT NULL,
        source_type TEXT NOT NULL,
        customers INTEGER,
        contribution_rate REAL,
        client_period_id INTEGER,
        FOREIGN KEY (client_period_id) REFERENCES clients_periods (id)
    );
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def list_tables(conn):
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        return [table[0] for table in tables]
    except sqlite3.Error as e:
        print("Error:", e)
        return []

conn = create_connection("/home/cristi/Documents/practica/database/ClientDB/Clients.db")
create_client_periods_table(conn)
create_records_table(conn)
tables = list_tables(conn)
for table in tables:
    print(table)

clients_periods
sqlite_sequence
records


In [210]:
def insert_to_clients_periods(conn, data):
    insertion_str = """INSERT INTO clients_periods (period_end, client_id, attribution, total_customers, contribution_rate) VALUES (?, ?, ?, ?, ?)"""
    cur = conn.cursor()
    cur.executemany(insertion_str, data)
    conn.commit()


In [211]:
def insert_to_records(conn, data):
    insertion_str="""INSERT INTO records (source, source_type, customers, contribution_rate, client_period_id) VALUES (?, ?, ?, ?, ?)"""
    cur = conn.cursor()
    cur.executemany(insertion_str, data)
    conn.commit()

In [212]:
def load_data_from_json(conn):
    client_data = []
    record_data=[]
    index=0
    with open("/home/cristi/Documents/practica/clients_per_day.json", 'r') as f:
        client_json = json.load(f)
    for client_id in client_json:
        for period in client_json[client_id]:
            date = period
            attribution = client_json[client_id][period]["attribution"]
            customers = client_json[client_id][period]["customers"]
            cr = client_json[client_id][period]["CR"]
            client_data.append((date, client_id, attribution, customers, cr))  # Reorder the values
            index+=1
            for record_dict in client_json[client_id][period]["records"]:
                source=record_dict["source"]
                source_type=record_dict["source_type"]
                attribution=record_dict["attribution"]
                cr=record_dict["CR"]
                record_data.append((source,source_type, attribution,cr, index))
    return client_data, record_data


In [213]:
clients_periods_table, records_table=load_data_from_json(conn)
# insert_to_clients_periods(conn,clients_periods_table)
# insert_to_records(conn, records_table)